# Rapport Détaillé : Dimensionnement d'un Buffer Réseau

<div align="center">

## Projet de Modélisation par Chaînes de Markov à Temps Discret

**Cours :** File d'attente, TSP FISA 2A  
**Auteur :** Analyse de performance réseau  
**Date :** 2024

---

[![Python](https://img.shields.io/badge/Python-3.7+-blue.svg)](https://www.python.org/)
[![License](https://img.shields.io/badge/License-MIT-green.svg)](LICENSE)
[![Jupyter](https://img.shields.io/badge/Jupyter-Notebook-orange.svg)](https://jupyter.org/)

</div>

---

## Table des Matières

1. [Guide d'Affichage](#affichage)
2. [Introduction et Contexte](#introduction)
3. [Modélisation du Système](#modelisation)
4. [Question 1 : Chaîne de Markov](#q1)
5. [Question 2 : Distribution Stationnaire](#q2)
6. [Questions 3-7 : Métriques de Performance](#q3-q7)
7. [Question 8 : Analyse Multi-Capacités](#q8)
8. [Question 9 : Analyse des Résultats](#q9)
9. [Question 10 : Comparaison de Configurations](#q10)
10. [Conclusion et Recommandations](#conclusion)

---

<a id="affichage"></a>
## Guide d'Affichage du Rapport

### Prérequis

Pour afficher correctement ce rapport, vous devez avoir installé les dépendances suivantes :

```bash
pip install numpy scipy matplotlib networkx pandas jupyter
```

### Affichage dans Jupyter Notebook

1. **Ouvrir le notebook** : Lancez Jupyter Notebook ou JupyterLab
   ```bash
   jupyter notebook Rapport_Projet_Buffer.ipynb
   ```
   ou
   ```bash
   jupyter lab Rapport_Projet_Buffer.ipynb
   ```

2. **Exécuter les cellules** : 
   - Exécutez toutes les cellules dans l'ordre (Menu : `Cell` → `Run All`)
   - Ou exécutez cellule par cellule avec `Shift + Enter`

3. **Affichage des formules mathématiques** :
   - Les formules LaTeX sont automatiquement rendues dans Jupyter
   - Assurez-vous que MathJax est activé (par défaut dans Jupyter)

### Export du Rapport

Pour exporter le rapport en différents formats :

- **HTML** : `File` → `Download as` → `HTML (.html)`
- **PDF** : `File` → `Download as` → `PDF via LaTeX (.pdf)` (nécessite LaTeX installé)
- **Markdown** : `File` → `Download as` → `Markdown (.md)`

### Note Importante

Les graphiques sont générés dynamiquement lors de l'exécution. Assurez-vous d'exécuter toutes les cellules pour voir tous les résultats et visualisations.

---


In [ ]:
# Configuration et imports
import numpy as np
from scipy.linalg import solve
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import style
import networkx as nx
import pandas as pd
from IPython.display import display, HTML, Image
import warnings
warnings.filterwarnings('ignore')

# Configuration du style matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10

# Couleurs personnalisées pour les graphiques
COLORS = {
    'primary': '#2E86AB',
    'secondary': '#A23B72',
    'success': '#06A77D',
    'warning': '#F18F01',
    'danger': '#C73E1D',
    'info': '#6C5CE7'
}

print("Bibliothèques importées avec succès !")
print("Configuration des graphiques appliquée")


<a id="introduction"></a>
## 2. Introduction et Contexte

### Objectif du Projet

Ce projet vise à **dimensionner un buffer** dans un nœud réseau en utilisant une **modélisation par chaînes de Markov à temps discret**. L'objectif est d'analyser les performances du système en fonction de différentes capacités de buffer et distributions d'arrivées de paquets.

### Contexte Réseau

Le système modélise un **nœud réseau (n1)** qui :
- Reçoit des paquets depuis d'autres nœuds
- Stocke temporairement les paquets dans un **buffer de capacité finie K**
- Transmet les paquets vers un autre nœud (n2) avec un service déterministe

### Paramètres du Système

| Paramètre | Valeur | Description |
|-----------|--------|-------------|
| **Taille des paquets** | 64 octets | Taille fixe des paquets |
| **Débit du lien** | 512 Mbits/s | Capacité de transmission |
| **Unité de temps** | 1 slot = 10⁻⁶ s | Microseconde |
| **Service** | 1 paquet/slot | Service déterministe |
| **Capacité buffer** | K (finie) | Variable à optimiser |

### Distribution d'Arrivées

#### Configuration Originale
- **p₀ = 0.4** : Probabilité de 0 arrivée par slot
- **p₁ = 0.2** : Probabilité de 1 arrivée par slot  
- **p₂ = 0.4** : Probabilité de 2 arrivées par slot

#### Configuration Modifiée (Question 10)
- **p₀ = 0.5** : Probabilité de 0 arrivée par slot
- **p₁ = 0.2** : Probabilité de 1 arrivée par slot
- **p₃ = 0.3** : Probabilité de 3 arrivées par slot


<a id="modelisation"></a>
## 3. Modélisation du Système

### Principe de la Chaîne de Markov

Une **chaîne de Markov à temps discret** est utilisée pour modéliser l'évolution du nombre de paquets dans le buffer au cours du temps.

#### États du Système
- **État i** : Nombre de paquets dans le buffer (i ∈ {0, 1, 2, ..., K})
- **K+1 états** possibles au total

#### Transitions d'État

Pour chaque état **i**, le système évolue selon :

1. **Service** : 1 paquet est servi (si i > 0)
   - État après service : $\max(0, i-1)$

2. **Arrivées** : 0, 1, 2 ou 3 paquets arrivent selon les probabilités
   - Nouvel état : $\min(K, \text{état\_après\_service} + \text{arrivées})$
   - Les paquets en excès sont **rejetés** si le buffer est plein

#### Matrice de Transition P

La matrice **P** de taille $(K+1) \times (K+1)$ représente les probabilités de transition :

$$P_{ij} = \mathbb{P}(X_{n+1} = j \mid X_n = i)$$

où $X_n$ est l'état du système au slot $n$.

### Métriques de Performance

Le projet calcule les métriques suivantes :

#### Distribution Stationnaire π

La distribution stationnaire $\pi = [\pi_0, \pi_1, \ldots, \pi_K]$ est obtenue en résolvant :

$$\pi = \pi P \quad \text{avec} \quad \sum_{i=0}^{K} \pi_i = 1$$

#### Nombre Moyen de Clients L

$$L = \sum_{i=0}^{K} i \cdot \pi_i$$

#### Débit d'Entrée Xe

$$X_e = \sum_{i=0}^{K} \pi_i \cdot \mathbb{E}[\text{paquets acceptés} \mid \text{état } i]$$

#### Taux de Perte Loss

$$\text{Loss} = \sum_{i=0}^{K} \pi_i \cdot \mathbb{E}[\text{paquets perdus} \mid \text{état } i]$$

#### Temps de Réponse Moyen R (Formule de Little)

$$R = \frac{L}{X_e}$$

#### Taux d'Utilisation U

$$U = 1 - \pi_0$$

Le taux d'utilisation représente la probabilité que le buffer soit non vide, c'est-à-dire que le lien soit occupé.


In [ ]:
# Chargement des fonctions du projet
exec(open('buffer_dimensionnement.py').read())

print("Fonctions du projet chargées avec succès !")


<a id="q1"></a>
## 4. Question 1 : Chaîne de Markov

### Description de la Chaîne

Pour **K = 5**, la chaîne de Markov possède **6 états** (0 à 5) représentant le nombre de paquets dans le buffer.


In [ ]:
# Paramètres pour Q1
K_q1 = 5
p0, p1, p2 = 0.4, 0.2, 0.4

# Affichage de la description
afficher_chaine_markov(K=K_q1, p0=p0, p1=p1, p2=p2)

# Génération du graphe
print("\nGénération du graphe de la chaîne de Markov...")
graphe_chaine_markov(K=K_q1, p0=p0, p1=p1, p2=p2)

# Affichage du graphe
try:
    display(Image('graphes/Q1_chaine_markov_K5.png', width=800))
except:
    print("Le fichier graphique sera généré lors de l'exécution complète")


<a id="q2"></a>
## 5. Question 2 : Distribution de Probabilité Stationnaire π(K)

### Calcul de la Distribution Stationnaire

La distribution stationnaire $\pi = [\pi_0, \pi_1, \ldots, \pi_K]$ est obtenue en résolvant le système d'équations :

$$\pi = \pi P$$

avec la contrainte de normalisation :

$$\sum_{i=0}^{K} \pi_i = 1$$

Ce système peut être réécrit sous la forme :

$$(P^T - I) \pi = 0$$

où $I$ est la matrice identité. En remplaçant la dernière équation par la contrainte de normalisation, on obtient un système linéaire résoluble.

Cette distribution représente les **probabilités d'équilibre** de chaque état du système à long terme, c'est-à-dire :

$$\pi_i = \lim_{n \to \infty} \mathbb{P}(X_n = i)$$
pport

In [ ]:
# Analyse pour différentes valeurs de K
valeurs_K = [2, 5, 10, 15, 80]
resultats_original = []

print("="*70)
print("CALCUL DES DISTRIBUTIONS STATIONNAIRES")
print("="*70)

for K in valeurs_K:
    pi = calculer_distribution_stationnaire(K, p0, p1, p2)
    resultats_original.append({
        'K': K,
        'pi': pi
    })
    print(f"\nK = {K}")
    print(f"   Distribution π : {[f'{p:.4f}' for p in pi[:min(6, len(pi))]]}{'...' if len(pi) > 6 else ''}")

# Visualisation interactive
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for idx, res in enumerate(resultats_original):
    if idx >= len(axes):
        break
    
    K = res['K']
    pi = res['pi']
    etats = list(range(len(pi)))
    
    # Graphique en barres avec style moderne
    bars = axes[idx].bar(etats, pi, color=COLORS['primary'], alpha=0.8, 
                         edgecolor='white', linewidth=1.5)
    
    # Ajout de valeurs sur les barres (pour les petits K)
    if K <= 15:
        for i, (etat, prob) in enumerate(zip(etats, pi)):
            if prob > 0.01:  # Afficher seulement si significatif
                axes[idx].text(etat, prob + 0.01, f'{prob:.3f}', 
                              ha='center', va='bottom', fontsize=9, fontweight='bold')
    
    axes[idx].set_xlabel('État i (nombre de paquets)', fontsize=11, fontweight='bold')
    axes[idx].set_ylabel('Probabilité π[i]', fontsize=11, fontweight='bold')
    axes[idx].set_title(f'K = {K}', fontsize=13, fontweight='bold', 
                       color=COLORS['secondary'])
    axes[idx].grid(True, alpha=0.3, linestyle='--')
    axes[idx].set_xticks(etats[::max(1, len(etats)//10)])  # Limiter les ticks pour K=80

# Masquer les axes inutilisés
for idx in range(len(resultats_original), len(axes)):
    axes[idx].axis('off')

plt.suptitle('Q2 : Distribution de Probabilité Stationnaire π(K) - Configuration Originale',
             fontsize=16, fontweight='bold', y=0.995, color=COLORS['secondary'])
plt.tight_layout()
plt.show()

print("\nVisualisation générée avec succès !")


<a id="q3-q7"></a>
## 6. Questions 3-7 : Métriques de Performance

### Calcul des Métriques

Nous calculons maintenant toutes les métriques de performance pour chaque valeur de K :

#### Question 3 : Nombre Moyen de Clients L

$$L = \sum_{i=0}^{K} i \cdot \pi_i$$

Cette métrique représente le nombre moyen de paquets présents dans le système (buffer) en régime stationnaire.

#### Question 4 : Débit d'Entrée Xe

$$X_e = \sum_{i=0}^{K} \pi_i \cdot \sum_{a=0}^{A} p_a \cdot \min(a, K - \max(0, i-1))$$

où $A$ est le nombre maximum d'arrivées possibles (2 ou 3 selon la configuration) et $p_a$ est la probabilité d'avoir $a$ arrivées.

#### Question 5 : Nombre Moyen de Paquets Perdus Loss

$$\text{Loss} = \sum_{i=0}^{K} \pi_i \cdot \sum_{a=0}^{A} p_a \cdot \max(0, a - (K - \max(0, i-1)))$$

Les paquets sont perdus lorsque le buffer est plein et qu'il y a plus d'arrivées que d'espaces disponibles.

#### Question 6 : Temps de Réponse Moyen R (Formule de Little)

$$R = \frac{L}{X_e}$$

Cette formule fondamentale de la théorie des files d'attente relie le nombre moyen de clients dans le système, le débit d'entrée et le temps de réponse moyen.

#### Question 7 : Taux d'Utilisation du Lien U

$$U = 1 - \pi_0 = \sum_{i=1}^{K} \pi_i$$

Le taux d'utilisation représente la probabilité que le buffer soit non vide, c'est-à-dire que le lien de transmission soit occupé.


In [ ]:
# Calcul complet des métriques pour toutes les valeurs de K
resultats_complets = []

print("="*70)
print("CALCUL DES MÉTRIQUES DE PERFORMANCE (Q3-Q7)")
print("="*70)

for K in valeurs_K:
    resultat = analyser_buffer(K, p0, p1, p2, verbose=False)
    if resultat:
        resultats_complets.append(resultat)

# Création d'un DataFrame pour un affichage tabulaire élégant
df_resultats = pd.DataFrame({
    'K': [r['K'] for r in resultats_complets],
    'L (paquets)': [f"{r['L']:.6f}" for r in resultats_complets],
    'Xe (paquets/slot)': [f"{r['Xe']:.6f}" for r in resultats_complets],
    'Loss (paquets/slot)': [f"{r['Loss']:.6e}" for r in resultats_complets],
    'R (slots)': [f"{r['R']:.6f}" for r in resultats_complets],
    'U (%)': [f"{r['U']*100:.2f}" for r in resultats_complets]
})

print("\nTableau Récapitulatif des Métriques")
print("="*70)
display(HTML(df_resultats.to_html(index=False, escape=False, 
                                   classes='table table-striped table-hover',
                                   table_id='resultats_table')))

# Visualisation des métriques
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

K_values = [r['K'] for r in resultats_complets]
L_values = [r['L'] for r in resultats_complets]
Xe_values = [r['Xe'] for r in resultats_complets]
Loss_values = [r['Loss'] for r in resultats_complets]
R_values = [r['R'] for r in resultats_complets]
U_values = [r['U']*100 for r in resultats_complets]

# Q3 : Nombre moyen de clients L
axes[0, 0].plot(K_values, L_values, 'o-', linewidth=3, markersize=10, 
                color=COLORS['primary'], markerfacecolor='white', markeredgewidth=2)
axes[0, 0].fill_between(K_values, L_values, alpha=0.3, color=COLORS['primary'])
axes[0, 0].set_xlabel('Capacité K', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('L (paquets)', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Q3 : Nombre moyen de clients L', 
                    fontsize=13, fontweight='bold', color=COLORS['secondary'])
axes[0, 0].grid(True, alpha=0.3, linestyle='--')

# Q4 : Débit d'entrée Xe
axes[0, 1].plot(K_values, Xe_values, 's-', linewidth=3, markersize=10,
                color=COLORS['success'], markerfacecolor='white', markeredgewidth=2)
axes[0, 1].fill_between(K_values, Xe_values, alpha=0.3, color=COLORS['success'])
axes[0, 1].set_xlabel('Capacité K', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Xe (paquets/slot)', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Q4 : Débit d\'entrée Xe', 
                    fontsize=13, fontweight='bold', color=COLORS['secondary'])
axes[0, 1].grid(True, alpha=0.3, linestyle='--')

# Q5 : Taux de perte Loss
axes[0, 2].semilogy(K_values, Loss_values, '^-', linewidth=3, markersize=10,
                    color=COLORS['danger'], markerfacecolor='white', markeredgewidth=2)
axes[0, 2].set_xlabel('Capacité K', fontsize=12, fontweight='bold')
axes[0, 2].set_ylabel('Loss (paquets/slot)', fontsize=12, fontweight='bold')
axes[0, 2].set_title('Q5 : Nombre moyen de paquets perdus', 
                    fontsize=13, fontweight='bold', color=COLORS['secondary'])
axes[0, 2].grid(True, alpha=0.3, linestyle='--', which='both')

# Q6 : Temps de réponse moyen R
axes[1, 0].plot(K_values, R_values, 'd-', linewidth=3, markersize=10,
                color=COLORS['info'], markerfacecolor='white', markeredgewidth=2)
axes[1, 0].fill_between(K_values, R_values, alpha=0.3, color=COLORS['info'])
axes[1, 0].set_xlabel('Capacité K', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('R (slots)', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Q6 : Temps de réponse moyen R', 
                    fontsize=13, fontweight='bold', color=COLORS['secondary'])
axes[1, 0].grid(True, alpha=0.3, linestyle='--')

# Q7 : Taux d'utilisation U
axes[1, 1].plot(K_values, U_values, '*-', linewidth=3, markersize=12,
                color=COLORS['warning'], markerfacecolor='white', markeredgewidth=2)
axes[1, 1].fill_between(K_values, U_values, alpha=0.3, color=COLORS['warning'])
axes[1, 1].set_xlabel('Capacité K', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('U (%)', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Q7 : Taux d\'utilisation U', 
                    fontsize=13, fontweight='bold', color=COLORS['secondary'])
axes[1, 1].grid(True, alpha=0.3, linestyle='--')
axes[1, 1].set_ylim([0, 105])

# Vue d'ensemble normalisée
L_norm = np.array(L_values) / max(L_values) if max(L_values) > 0 else L_values
Xe_norm = np.array(Xe_values) / max(Xe_values) if max(Xe_values) > 0 else Xe_values
U_norm = np.array(U_values) / 100

axes[1, 2].plot(K_values, L_norm, 'o-', label='L (normalisé)', 
                linewidth=2.5, markersize=8, color=COLORS['primary'])
axes[1, 2].plot(K_values, Xe_norm, 's-', label='Xe (normalisé)', 
                linewidth=2.5, markersize=8, color=COLORS['success'])
axes[1, 2].plot(K_values, U_norm, '*-', label='U (normalisé)', 
                linewidth=2.5, markersize=10, color=COLORS['warning'])
axes[1, 2].set_xlabel('Capacité K', fontsize=12, fontweight='bold')
axes[1, 2].set_ylabel('Valeur normalisée', fontsize=12, fontweight='bold')
axes[1, 2].set_title('Vue d\'ensemble (normalisée)', 
                    fontsize=13, fontweight='bold', color=COLORS['secondary'])
axes[1, 2].legend(loc='best', framealpha=0.9, shadow=True)
axes[1, 2].grid(True, alpha=0.3, linestyle='--')

plt.suptitle('Questions Q3-Q7 : Métriques de Performance en fonction de K',
             fontsize=16, fontweight='bold', y=0.995, color=COLORS['secondary'])
plt.tight_layout()
plt.show()

print("\nVisualisations générées avec succès !")


<a id="q8"></a>
## 7. Question 8 : Analyse Multi-Capacités

### Analyse pour K ∈ {2, 5, 10, 15, 80}

Cette section présente une analyse comparative détaillée pour différentes capacités de buffer, incluant :
- L'état le plus probable pour chaque K
- Les probabilités que le buffer soit vide ou plein
- Une comparaison visuelle des métriques principales


In [ ]:
# Analyse détaillée pour chaque K
print("="*70)
print("ANALYSE DÉTAILLÉE PAR CAPACITÉ K")
print("="*70)

# Création d'un tableau détaillé avec toutes les informations
analyse_detaille = []

for res in resultats_complets:
    K = res['K']
    pi = res['pi']
    
    # Trouver l'état le plus probable
    etat_max_prob = np.argmax(pi)
    prob_max = pi[etat_max_prob]
    
    # Probabilité que le buffer soit plein
    prob_plein = pi[K]
    
    # Probabilité que le buffer soit vide
    prob_vide = pi[0]
    
    analyse_detaille.append({
        'K': K,
        'État le plus probable': etat_max_prob,
        'Prob. max': f"{prob_max:.4f}",
        'Prob. buffer vide (π[0])': f"{prob_vide:.4f}",
        'Prob. buffer plein (π[K])': f"{prob_plein:.6f}",
        'L': f"{res['L']:.4f}",
        'Xe': f"{res['Xe']:.4f}",
        'Loss': f"{res['Loss']:.6e}",
        'R': f"{res['R']:.4f}",
        'U (%)': f"{res['U']*100:.2f}"
    })

df_detaille = pd.DataFrame(analyse_detaille)
display(HTML(df_detaille.to_html(index=False, escape=False,
                                 classes='table table-striped table-hover',
                                 table_id='analyse_detaille_table')))

# Graphique comparatif avec barres groupées
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Graphique 1 : Métriques principales
x = np.arange(len(K_values))
width = 0.2

# Normalisation pour la visualisation
L_norm_viz = np.array(L_values) / max(L_values) * 100
Xe_norm_viz = np.array(Xe_values) / max(Xe_values) * 100

axes[0].bar(x - width, L_norm_viz, width, label='L (normalisé)', 
           color=COLORS['primary'], alpha=0.8)
axes[0].bar(x, Xe_norm_viz, width, label='Xe (normalisé)', 
           color=COLORS['success'], alpha=0.8)
axes[0].bar(x + width, U_values, width, label='U (%)', 
           color=COLORS['warning'], alpha=0.8)

axes[0].set_xlabel('Capacité K', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Valeur normalisée (%)', fontsize=12, fontweight='bold')
axes[0].set_title('Comparaison des Métriques Principales', 
                 fontsize=13, fontweight='bold', color=COLORS['secondary'])
axes[0].set_xticks(x)
axes[0].set_xticklabels(K_values)
axes[0].legend(loc='best', framealpha=0.9, shadow=True)
axes[0].grid(True, alpha=0.3, linestyle='--', axis='y')

# Graphique 2 : Taux de perte (échelle logarithmique)
axes[1].semilogy(K_values, Loss_values, 'o-', linewidth=3, markersize=12,
                color=COLORS['danger'], markerfacecolor='white', markeredgewidth=2)
axes[1].fill_between(K_values, Loss_values, alpha=0.3, color=COLORS['danger'])
axes[1].set_xlabel('Capacité K', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Loss (paquets/slot) - Échelle log', fontsize=12, fontweight='bold')
axes[1].set_title('Évolution du Taux de Perte', 
                 fontsize=13, fontweight='bold', color=COLORS['secondary'])
axes[1].grid(True, alpha=0.3, linestyle='--', which='both')

plt.tight_layout()
plt.show()

print("\nAnalyse multi-capacités terminée !")


<a id="q9"></a>
## 8. Question 9 : Analyse et Commentaires sur les Résultats

### Observations Clés

#### Impact de la Capacité K sur les Performances

1. **Nombre moyen de clients (L)**
   - Augmente avec K : Plus de capacité = plus de paquets stockés en moyenne
   - Tendance à se stabiliser pour de grandes valeurs de K

2. **Débit d'entrée (Xe)**
   - Augmente avec K : Moins de rejets = plus de paquets acceptés
   - Converge vers le débit d'arrivée théorique pour K → ∞

3. **Taux de perte (Loss)**
   - Diminue exponentiellement avec K
   - Pour K petit : pertes importantes dues au buffer plein
   - Pour K grand : pertes négligeables

4. **Temps de réponse (R)**
   - Augmente avec K : Plus de paquets = temps d'attente plus long
   - Trade-off entre pertes et délai

5. **Taux d'utilisation (U)**
   - Augmente avec K : Le lien est utilisé plus souvent
   - Reflète la probabilité que le buffer soit non vide


In [ ]:
# Analyse quantitative des tendances
print("="*70)
print("ANALYSE QUANTITATIVE DES TENDANCES")
print("="*70)

# Calcul des variations
variations = []
for i in range(len(resultats_complets) - 1):
    K_curr = resultats_complets[i]['K']
    K_next = resultats_complets[i+1]['K']
    
    L_curr = resultats_complets[i]['L']
    L_next = resultats_complets[i+1]['L']
    var_L = ((L_next - L_curr) / L_curr * 100) if L_curr > 0 else 0
    
    Xe_curr = resultats_complets[i]['Xe']
    Xe_next = resultats_complets[i+1]['Xe']
    var_Xe = ((Xe_next - Xe_curr) / Xe_curr * 100) if Xe_curr > 0 else 0
    
    Loss_curr = resultats_complets[i]['Loss']
    Loss_next = resultats_complets[i+1]['Loss']
    var_Loss = ((Loss_next - Loss_curr) / Loss_curr * 100) if Loss_curr > 0 else 0
    
    variations.append({
        'K': f"{K_curr} → {K_next}",
        'ΔL (%)': f"{var_L:+.2f}",
        'ΔXe (%)': f"{var_Xe:+.2f}",
        'ΔLoss (%)': f"{var_Loss:+.2f}"
    })

df_variations = pd.DataFrame(variations)
display(HTML(df_variations.to_html(index=False, escape=False,
                                   classes='table table-striped table-hover',
                                   table_id='variations_table')))

# Visualisation des tendances
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Tendance de L
axes[0, 0].plot(K_values, L_values, 'o-', linewidth=3, markersize=10,
               color=COLORS['primary'], markerfacecolor='white', markeredgewidth=2)
axes[0, 0].set_xlabel('Capacité K', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('L (paquets)', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Tendance : Nombre moyen de clients', 
                    fontsize=13, fontweight='bold', color=COLORS['secondary'])
axes[0, 0].grid(True, alpha=0.3, linestyle='--')
# Ajout d'une ligne de tendance
z = np.polyfit(K_values, L_values, 2)
p = np.poly1d(z)
axes[0, 0].plot(K_values, p(K_values), '--', color='red', alpha=0.5, 
               linewidth=2, label='Tendance polynomiale')
axes[0, 0].legend()

# Tendance de Xe
axes[0, 1].plot(K_values, Xe_values, 's-', linewidth=3, markersize=10,
               color=COLORS['success'], markerfacecolor='white', markeredgewidth=2)
axes[0, 1].set_xlabel('Capacité K', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Xe (paquets/slot)', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Tendance : Débit d\'entrée', 
                    fontsize=13, fontweight='bold', color=COLORS['secondary'])
axes[0, 1].grid(True, alpha=0.3, linestyle='--')
# Ligne asymptotique (débit théorique)
debit_theorique = p0*0 + p1*1 + p2*2  # Débit d'arrivée moyen
axes[0, 1].axhline(y=debit_theorique, color='red', linestyle='--', 
                  alpha=0.5, linewidth=2, label=f'Débit théorique = {debit_theorique:.2f}')
axes[0, 1].legend()

# Tendance de Loss (log)
axes[1, 0].semilogy(K_values, Loss_values, '^-', linewidth=3, markersize=10,
                    color=COLORS['danger'], markerfacecolor='white', markeredgewidth=2)
axes[1, 0].set_xlabel('Capacité K', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Loss (paquets/slot)', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Tendance : Taux de perte (décroissance exponentielle)', 
                    fontsize=13, fontweight='bold', color=COLORS['secondary'])
axes[1, 0].grid(True, alpha=0.3, linestyle='--', which='both')

# Tendance de R
axes[1, 1].plot(K_values, R_values, 'd-', linewidth=3, markersize=10,
               color=COLORS['info'], markerfacecolor='white', markeredgewidth=2)
axes[1, 1].set_xlabel('Capacité K', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('R (slots)', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Tendance : Temps de réponse moyen', 
                    fontsize=13, fontweight='bold', color=COLORS['secondary'])
axes[1, 1].grid(True, alpha=0.3, linestyle='--')

plt.suptitle('Question 9 : Analyse des Tendances en fonction de K',
             fontsize=16, fontweight='bold', y=0.995, color=COLORS['secondary'])
plt.tight_layout()
plt.show()

print("\nObservations :")
print(f"   - Débit théorique d'arrivée : {debit_theorique:.4f} paquets/slot")
print(f"   - Débit d'entrée pour K=80 : {Xe_values[-1]:.4f} paquets/slot")
print(f"   - Taux de perte pour K=2 : {Loss_values[0]:.6e} paquets/slot")
print(f"   - Taux de perte pour K=80 : {Loss_values[-1]:.6e} paquets/slot")
print(f"   - Réduction du taux de perte : {(1 - Loss_values[-1]/Loss_values[0])*100:.2f}%")


<a id="q10"></a>
## 9. Question 10 : Comparaison de Configurations

### Configuration Modifiée

Nous comparons maintenant les résultats avec une **nouvelle distribution d'arrivées** :

| Configuration | p₀ | p₁ | p₂ | p₃ |
|---------------|----|----|----|----|
| **Originale** | 0.4 | 0.2 | 0.4 | 0.0 |
| **Modifiée** | 0.5 | 0.2 | 0.0 | 0.3 |

### Changements Clés

- **p₀ augmente** : Plus de slots sans arrivée (0.4 → 0.5)
- **p₂ diminue** : Moins d'arrivées de 2 paquets (0.4 → 0.0)
- **p₃ introduit** : Nouveaux pics d'arrivée de 3 paquets (0.0 → 0.3)

### Impact Attendu

- **Pics plus importants** : Arrivées de 3 paquets peuvent saturer le buffer
- **Plus de périodes creuses** : p₀ plus élevé permet au buffer de se vider
- **Trade-off** : Entre pics intenses et périodes de récupération


In [ ]:
# Nouvelles probabilités
p0_new, p1_new, p2_new, p3_new = 0.5, 0.2, 0.0, 0.3

# Analyse avec les nouvelles probabilités
print("="*70)
print("ANALYSE AVEC CONFIGURATION MODIFIÉE")
print("="*70)

resultats_nouvelle = []
for K in valeurs_K:
    resultat = analyser_buffer(K, p0_new, p1_new, p2_new, p3_new, verbose=False)
    if resultat:
        resultats_nouvelle.append(resultat)

# Tableau comparatif
df_comparaison = pd.DataFrame({
    'K': [r['K'] for r in resultats_complets],
    'L (orig)': [f"{r['L']:.4f}" for r in resultats_complets],
    'L (new)': [f"{r['L']:.4f}" for r in resultats_nouvelle],
    'Xe (orig)': [f"{r['Xe']:.4f}" for r in resultats_complets],
    'Xe (new)': [f"{r['Xe']:.4f}" for r in resultats_nouvelle],
    'Loss (orig)': [f"{r['Loss']:.6e}" for r in resultats_complets],
    'Loss (new)': [f"{r['Loss']:.6e}" for r in resultats_nouvelle],
    'R (orig)': [f"{r['R']:.4f}" for r in resultats_complets],
    'R (new)': [f"{r['R']:.4f}" for r in resultats_nouvelle],
    'U (orig) %': [f"{r['U']*100:.2f}" for r in resultats_complets],
    'U (new) %': [f"{r['U']*100:.2f}" for r in resultats_nouvelle]
})

display(HTML(df_comparaison.to_html(index=False, escape=False,
                                    classes='table table-striped table-hover',
                                    table_id='comparaison_table')))

# Extraction des valeurs pour visualisation
L_orig = [r['L'] for r in resultats_complets]
Xe_orig = [r['Xe'] for r in resultats_complets]
Loss_orig = [r['Loss'] for r in resultats_complets]
R_orig = [r['R'] for r in resultats_complets]
U_orig = [r['U']*100 for r in resultats_complets]

L_new = [r['L'] for r in resultats_nouvelle]
Xe_new = [r['Xe'] for r in resultats_nouvelle]
Loss_new = [r['Loss'] for r in resultats_nouvelle]
R_new = [r['R'] for r in resultats_nouvelle]
U_new = [r['U']*100 for r in resultats_nouvelle]


In [ ]:
# Visualisation comparative
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# L : Nombre moyen de clients
axes[0, 0].plot(K_values, L_orig, 'o-', linewidth=3, markersize=10,
               label='Original (p0=0.4, p1=0.2, p2=0.4)', 
               color=COLORS['primary'], markerfacecolor='white', markeredgewidth=2)
axes[0, 0].plot(K_values, L_new, 's--', linewidth=3, markersize=10,
               label='Nouveau (p0=0.5, p1=0.2, p3=0.3)', 
               color=COLORS['danger'], markerfacecolor='white', markeredgewidth=2)
axes[0, 0].set_xlabel('Capacité K', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('L (paquets)', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Q10 : Nombre moyen de clients L', 
                    fontsize=13, fontweight='bold', color=COLORS['secondary'])
axes[0, 0].legend(loc='best', framealpha=0.9, shadow=True)
axes[0, 0].grid(True, alpha=0.3, linestyle='--')

# Xe : Débit d'entrée
axes[0, 1].plot(K_values, Xe_orig, 'o-', linewidth=3, markersize=10,
               label='Original', color=COLORS['primary'], 
               markerfacecolor='white', markeredgewidth=2)
axes[0, 1].plot(K_values, Xe_new, 's--', linewidth=3, markersize=10,
               label='Nouveau', color=COLORS['danger'], 
               markerfacecolor='white', markeredgewidth=2)
axes[0, 1].set_xlabel('Capacité K', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Xe (paquets/slot)', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Q10 : Débit d\'entrée Xe', 
                    fontsize=13, fontweight='bold', color=COLORS['secondary'])
axes[0, 1].legend(loc='best', framealpha=0.9, shadow=True)
axes[0, 1].grid(True, alpha=0.3, linestyle='--')

# Loss : Taux de perte
axes[0, 2].semilogy(K_values, Loss_orig, 'o-', linewidth=3, markersize=10,
                   label='Original', color=COLORS['primary'], 
                   markerfacecolor='white', markeredgewidth=2)
axes[0, 2].semilogy(K_values, Loss_new, 's--', linewidth=3, markersize=10,
                   label='Nouveau', color=COLORS['danger'], 
                   markerfacecolor='white', markeredgewidth=2)
axes[0, 2].set_xlabel('Capacité K', fontsize=12, fontweight='bold')
axes[0, 2].set_ylabel('Loss (paquets/slot)', fontsize=12, fontweight='bold')
axes[0, 2].set_title('Q10 : Taux de perte Loss', 
                    fontsize=13, fontweight='bold', color=COLORS['secondary'])
axes[0, 2].legend(loc='best', framealpha=0.9, shadow=True)
axes[0, 2].grid(True, alpha=0.3, linestyle='--', which='both')

# R : Temps de réponse
axes[1, 0].plot(K_values, R_orig, 'o-', linewidth=3, markersize=10,
               label='Original', color=COLORS['primary'], 
               markerfacecolor='white', markeredgewidth=2)
axes[1, 0].plot(K_values, R_new, 's--', linewidth=3, markersize=10,
               label='Nouveau', color=COLORS['danger'], 
               markerfacecolor='white', markeredgewidth=2)
axes[1, 0].set_xlabel('Capacité K', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('R (slots)', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Q10 : Temps de réponse moyen R', 
                    fontsize=13, fontweight='bold', color=COLORS['secondary'])
axes[1, 0].legend(loc='best', framealpha=0.9, shadow=True)
axes[1, 0].grid(True, alpha=0.3, linestyle='--')

# U : Taux d'utilisation
axes[1, 1].plot(K_values, U_orig, 'o-', linewidth=3, markersize=10,
               label='Original', color=COLORS['primary'], 
               markerfacecolor='white', markeredgewidth=2)
axes[1, 1].plot(K_values, U_new, 's--', linewidth=3, markersize=10,
               label='Nouveau', color=COLORS['danger'], 
               markerfacecolor='white', markeredgewidth=2)
axes[1, 1].set_xlabel('Capacité K', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('U (%)', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Q10 : Taux d\'utilisation U', 
                    fontsize=13, fontweight='bold', color=COLORS['secondary'])
axes[1, 1].legend(loc='best', framealpha=0.9, shadow=True)
axes[1, 1].grid(True, alpha=0.3, linestyle='--')
axes[1, 1].set_ylim([0, 105])

# Ratio nouveau/original
ratio_L = [new/orig if orig > 0 else 0 for new, orig in zip(L_new, L_orig)]
ratio_Xe = [new/orig if orig > 0 else 0 for new, orig in zip(Xe_new, Xe_orig)]
ratio_U = [new/orig if orig > 0 else 0 for new, orig in zip(U_new, U_orig)]

axes[1, 2].plot(K_values, ratio_L, 'o-', label='L (ratio)', 
               linewidth=2.5, markersize=8, color=COLORS['primary'])
axes[1, 2].plot(K_values, ratio_Xe, 's-', label='Xe (ratio)', 
               linewidth=2.5, markersize=8, color=COLORS['success'])
axes[1, 2].plot(K_values, ratio_U, '*-', label='U (ratio)', 
               linewidth=2.5, markersize=10, color=COLORS['warning'])
axes[1, 2].axhline(y=1, color='black', linestyle='--', linewidth=2, alpha=0.5)
axes[1, 2].set_xlabel('Capacité K', fontsize=12, fontweight='bold')
axes[1, 2].set_ylabel('Ratio (Nouveau/Original)', fontsize=12, fontweight='bold')
axes[1, 2].set_title('Q10 : Ratio Nouveau/Original', 
                    fontsize=13, fontweight='bold', color=COLORS['secondary'])
axes[1, 2].legend(loc='best', framealpha=0.9, shadow=True)
axes[1, 2].grid(True, alpha=0.3, linestyle='--')

plt.suptitle('Q10 : Comparaison Configuration Originale vs Nouvelle',
             fontsize=16, fontweight='bold', y=0.995, color=COLORS['secondary'])
plt.tight_layout()
plt.show()

print("\nComparaison terminée !")


In [ ]:
# Analyse quantitative des différences
print("="*70)
print("ANALYSE QUANTITATIVE DES DIFFÉRENCES")
print("="*70)

differences = []
for i, K in enumerate(K_values):
    diff_L = ((L_new[i] - L_orig[i]) / L_orig[i] * 100) if L_orig[i] > 0 else 0
    diff_Xe = ((Xe_new[i] - Xe_orig[i]) / Xe_orig[i] * 100) if Xe_orig[i] > 0 else 0
    diff_Loss = ((Loss_new[i] - Loss_orig[i]) / Loss_orig[i] * 100) if Loss_orig[i] > 0 else 0
    diff_R = ((R_new[i] - R_orig[i]) / R_orig[i] * 100) if R_orig[i] > 0 else 0
    diff_U = ((U_new[i] - U_orig[i]) / U_orig[i] * 100) if U_orig[i] > 0 else 0
    
    differences.append({
        'K': K,
        'ΔL (%)': f"{diff_L:+.2f}",
        'ΔXe (%)': f"{diff_Xe:+.2f}",
        'ΔLoss (%)': f"{diff_Loss:+.2f}",
        'ΔR (%)': f"{diff_R:+.2f}",
        'ΔU (%)': f"{diff_U:+.2f}"
    })

df_differences = pd.DataFrame(differences)
display(HTML(df_differences.to_html(index=False, escape=False,
                                   classes='table table-striped table-hover',
                                   table_id='differences_table')))

# Calcul du débit théorique pour les deux configurations
debit_theorique_orig = p0*0 + p1*1 + p2*2
debit_theorique_new = p0_new*0 + p1_new*1 + p3_new*3

print(f"\nDébits théoriques d'arrivée :")
print(f"   - Configuration originale : {debit_theorique_orig:.4f} paquets/slot")
print(f"   - Configuration nouvelle  : {debit_theorique_new:.4f} paquets/slot")
print(f"   - Différence              : {debit_theorique_new - debit_theorique_orig:+.4f} paquets/slot")

print(f"\nObservations principales :")
print(f"   - La nouvelle configuration a un débit d'arrivée théorique plus élevé")
print(f"   - Les pics de 3 paquets créent plus de congestion pour K petit")
print(f"   - Les périodes creuses (p0=0.5) permettent une meilleure récupération")
print(f"   - Pour K grand, l'impact des pics est atténué")


<a id="conclusion"></a>
## 10. Conclusion et Recommandations

### Synthèse des Résultats

#### Impact de la Capacité K

1. **Petites capacités (K = 2, 5)**
   - Taux de perte élevé
   - Débit d'entrée limité
   - Temps de réponse faible (peu de paquets en attente)
   - **Recommandation** : Inadapté pour des applications sensibles aux pertes

2. **Capacités moyennes (K = 10, 15)**
   - Bon compromis entre pertes et délai
   - Débit d'entrée proche du théorique
   - Temps de réponse acceptable
   - **Recommandation** : Optimal pour la plupart des applications

3. **Grandes capacités (K = 80)**
   - Taux de perte négligeable
   - Débit d'entrée maximal
   - Temps de réponse plus élevé
   - **Recommandation** : Adapté pour applications tolérantes au délai mais sensibles aux pertes

#### Comparaison des Configurations

- **Configuration originale** : Plus stable, moins de pics
- **Configuration modifiée** : Pics plus intenses mais périodes de récupération plus longues
- **Recommandation** : Choisir selon les contraintes de l'application

### Recommandations Finales

1. **Pour minimiser les pertes** : Utiliser K ≥ 15
2. **Pour minimiser le délai** : Utiliser K ≤ 10
3. **Pour un compromis optimal** : K = 10-15
4. **Pour la configuration** : Préférer l'originale pour plus de stabilité

---

<div align="center">

### Projet réalisé dans le cadre du cours de File d'attente, TSP FISA 2A

**Auteurs :**  
Nassim EL HADDAD  
Rebecca RINGUET

**Merci de votre attention !**

</div>
